# ML Term Paper - Code: Binary Models 
by Ann-Christin and Sarah 02/03/2021

## 1. Preparation

In [195]:
# import libraries
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import Lasso
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import datasets
from sklearn.preprocessing import StandardScaler


from sklearn import ensemble
from sklearn import tree as tree

# set color theme
sns.set_theme()

#pd.set_option("display.max_rows", None, "display.max_columns", None)

In [2]:
import os
# path = "E:\Data\krea\PIAAC" #enter your path here
path = "C:/Users/gust/Documents/ML_term_paper"
os.chdir(path)
print("done")
data = pd.read_csv("data/piaac_red.csv", sep=',', error_bad_lines=False, index_col=False)
data.shape

done


C:\Users\gust\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (1,2,5,12,18,23,25,26,29,31,32,39,48,51,58,59,60,63,64,66,69,76,85,86,87,93) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(230691, 130)

In [172]:
# data cleaning, drop variables that have too few observations,are not relevant or collinear
dcl = data.drop(columns = ['Unnamed: 0', 'cntry', 'cntry_e', 'inpiaac', "racethn_5cat", "unempflag", "racethn_5cat", "c_q08c2", "seqid", "b_q04b", "b_q19a", 
                         "c_q08c2", "learnatwork", "nfehrsnjr", "nfehrsjr", "nfehrs", "earnmthselfppp", "d_q05b2", 
                          "yrsqual_t", "earnmthbonusppp",  "d_q16d5", "c_q02c", "isco1c", "c_d09", "isic1l", "cnt_h", "isced_hf", "isco1l", "isco1c", "c_d05",
                          'icthome','ictwork','b_q01a','b_q01a_t','yrsqual','ageg10lfs','cnt_brth','fnfaet12jr', 'fe12', 'aetpop', 'faet12', 'faet12jr', 'faet12njr', 
                           'nfe12', 'nfe12jr', 'nfe12njr', 'fnfaet12', 'fnfaet12jr', 'fnfaet12njr']) # exlude those to test the results 
#"isic2l", "isic2c", "isic_cus_c", "isic_lus_c", "isic4_c", "isic4_l",

# further exclude 
dcl = dcl.drop(columns = ['d_q16b', 'b_q20b', 'b_d12h', 'b_q02a_t2', 'b_q11', 'b_q12e', 'd_q12a' , 'b_q04a', 'b_q16', 'b_q26b', 'b_q15c', 'd_q06c', 'b_q15b', 'd_q05a2', 
                          'b_q13', 'b_q15a', 'b_q16', 'b_q02a', 'b_q02a_t1', 'b_q05c', 'b_q10b', 'c_d06', 'd_q04', 'b_q12g', 'b_q12b', 'b_q12d'])


dcl = dcl.loc[:, ~dcl.columns.str.endswith('_c')]
#drop if skill level is missing 
#dcl = dcl[dcl.iscoskil4 != "A"]
#dcl = dcl[dcl.iscoskil4 != "N"]
#dcl = dcl[dcl.iscoskil4 != "U"]

In [173]:
#dcl['d_q05a2'] = dcl['d_q05a2'].astype('object')
dcl['gender_r'] = dcl['gender_r'].astype('category')
dcl["age_r"] = pd.to_numeric(dcl["age_r"])
dcl['b_q12a'] = dcl['b_q12a'].astype('category') # yes/no open educational training
#dcl['b_q12b'] = pd.to_numeric(dcl["b_q12b"]) # number of open/distance educational trainings
#dcl["b_q12d"] = pd.to_numeric(dcl["b_q12d"]) # number of on-the-job training
#dcl['b_q12e'] = dcl['b_q12e'].astype('category')  # yes/no seminars training
dcl['b_q12f'] = pd.to_numeric(dcl["b_q12f"]) # number of seminars
#dcl['b_q12g'] = dcl['b_q12g'].astype('category')  # yes/no private training
dcl['b_q12h'] = pd.to_numeric(dcl["b_q12h"]) # number of private educational trainings
# dcl['j_q03a'] = dcl['j_q03a'].astype('category') # yes/no children
dcl['j_q03b'] = pd.to_numeric(dcl['j_q03b']) # number of children
#dcl['yrsqual_t'] = pd.to_numeric(dcl['yrsqual_t'])
dcl['yrsget'] = pd.to_numeric(dcl['yrsget'])
dcl['iscoskil4'] = dcl['iscoskil4'].astype('category')
#dcl['h_q05g'] = dcl['h_q05g'].astype('category') # Skill use everyday life - ICT - Computer - How often - Programming language  
#dcl['g_q05g'] = dcl['g_q05g'].astype('category') # skill use work - ICT programming lanugage how often
#dcl['g_q05'] = dcl['g_q05'].astype('category') # use computer at work yes/no

In [174]:
cate = dcl.select_dtypes(include=['object', 'category'])
cate = cate.drop(columns = ["iscoskil4"]) 
cate = list(set(cate))
cate

['d_q06b',
 'd_q03',
 'isic1c',
 'd_q12b',
 'b_q12a',
 'd_q09',
 'g_q06',
 'b_q12c',
 'b_q14b',
 'g_q05d',
 'vet',
 'gender_r',
 'g_q04',
 'g_q05f',
 'd_q13c',
 'd_q14',
 'b_q26a_t',
 'd_q06a',
 'g_q05a',
 'g_q05h',
 'g_q07',
 'pared',
 'nopaidworkever',
 'g_q05c',
 'j_q04a',
 'b_q10a',
 'd_q12c',
 'f_q07b',
 'computerexperience',
 'd_q07a',
 'edcat8',
 'f_q07a',
 'g_q08',
 'd_q04_t',
 'leaver1624',
 'b_q14a',
 'b_q01b',
 'c_q02a',
 'b_q10c',
 'g_q05g',
 'g_q05e']

In [175]:
# generate dummies                
dummies1 = [pd.get_dummies(dcl[i], prefix=i, drop_first = True) for i in [cate]]
dummies2 =  [pd.get_dummies(dcl["iscoskil4"], prefix="skill")] 
dummies1 = dummies1.pop(0)
dummies1

,d_q06b_Increased,d_q06b_Stayed more or less the same,d_q03_The private sector (for example a company),d_q03_The public sector (for example the local government or a state school),isic1c_9995,isic1c_9996,isic1c_9997,isic1c_9998,isic1c_9999,isic1c_A,...,b_q10c_Somewhat useful,b_q10c_Very useful,g_q05g_Every day,g_q05g_Less than once a month,g_q05g_Less than once a week but at least once a month,g_q05g_Never,g_q05e_Every day,g_q05e_Less than once a month,g_q05e_Less than once a week but at least once a month,g_q05e_Never
0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
2,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230686,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
230687,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
230688,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0
230689,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1


In [176]:
# generate dummies                
dummies1 = [pd.get_dummies(dcl[i], prefix=i, drop_first = True) for i in [cate]]
dummies2 =  [pd.get_dummies(dcl["iscoskil4"], prefix="skill")] 
dummies1 = dummies1.pop(0)
dummies1 = pd.DataFrame(dummies1)
dummies2 = dummies2.pop(0)
dummies2 = pd.DataFrame(dummies2)
dummies = dummies1.join(dummies2)
dummies = dummies.drop(columns = ["skill_4","skill_U", "skill_A", "skill_N"])
dummies.head()
dummies.info()  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230691 entries, 0 to 230690
Columns: 137 entries, d_q06b_Increased to skill_3
dtypes: uint8(137)
memory usage: 30.1 MB


In [177]:
num = dcl.select_dtypes(include=['float64', "int32"])
num = num.loc[:, ~num.columns.str.startswith('b_q12')]

In [178]:
num.head()
num.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230691 entries, 0 to 230690
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   age_r          166949 non-null  float64
 1   j_q03b         103091 non-null  float64
 2   yrsget         125736 non-null  float64
 3   c_q09          160575 non-null  float64
 4   c_q10a         147420 non-null  float64
 5   readytolearn   226734 non-null  float64
 6   earnmthallppp  88508 non-null   float64
dtypes: float64(7)
memory usage: 12.3 MB


In [179]:
num_list = list(set(num))
num_list

['earnmthallppp',
 'c_q10a',
 'yrsget',
 'c_q09',
 'age_r',
 'readytolearn',
 'j_q03b']

In [180]:
# define X and y
on_job = dummies["b_q12c_Yes"] #define y here
open_educ = dummies["b_q12a_Yes"]
X = num.join(dummies)
X = pd.DataFrame(np.ascontiguousarray(X.values), columns = X.columns)
X = X.drop(columns = ["b_q12c_Yes", "b_q12a_Yes"])
y = pd.DataFrame(dict(on_job=on_job, open_educ =open_educ))
y_X = y.join(X)
y_X

,on_job,open_educ,age_r,j_q03b,yrsget,c_q09,c_q10a,readytolearn,earnmthallppp,d_q06b_Increased,...,g_q05g_Less than once a month,g_q05g_Less than once a week but at least once a month,g_q05g_Never,g_q05e_Every day,g_q05e_Less than once a month,g_q05e_Less than once a week but at least once a month,g_q05e_Never,skill_1,skill_2,skill_3
0,0,0,NaN,NaN,NaN,NaN,NaN,1.016017,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1,NaN,NaN,19.0,NaN,NaN,2.164922,NaN,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2,0,0,NaN,NaN,NaN,NaN,NaN,1.177736,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,0,NaN,NaN,NaN,NaN,NaN,2.112932,NaN,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,0,1,NaN,NaN,NaN,NaN,NaN,3.064464,NaN,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230686,0,0,NaN,NaN,8.0,NaN,NaN,1.177736,NaN,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
230687,0,0,NaN,NaN,11.0,NaN,NaN,2.156445,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
230688,1,0,NaN,NaN,11.0,NaN,NaN,2.383062,NaN,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
230689,0,0,NaN,NaN,12.0,NaN,NaN,1.751830,NaN,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


## 2. Logit

### 2.1 Logit 'on_job'

In [181]:
import statsmodels.api as sm 

nona = y_X.drop(columns = ["open_educ","isic1c_9995", "isic1c_9996", "isic1c_9997", "isic1c_9998", "isic1c_9999"])
nona = nona.dropna()
nona = nona.loc[:, (nona != 0).any(axis=0)]
y = nona["on_job"]
X = nona.iloc[:, 1:]


In [182]:
X

,age_r,j_q03b,yrsget,c_q09,c_q10a,readytolearn,earnmthallppp,d_q06b_Increased,d_q06b_Stayed more or less the same,d_q03_The private sector (for example a company),...,g_q05g_Less than once a month,g_q05g_Less than once a week but at least once a month,g_q05g_Never,g_q05e_Every day,g_q05e_Less than once a month,g_q05e_Less than once a week but at least once a month,g_q05e_Never,skill_1,skill_2,skill_3
18418,33.0,2.0,11.0,6.0,1.0,2.361800,2500.0000,0.0,1.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
18423,43.0,1.0,11.0,24.0,1.0,1.124230,1343.1400,0.0,1.0,1.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
18433,53.0,2.0,9.0,27.0,2.0,0.788280,980.3900,0.0,0.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
18434,42.0,1.0,11.0,18.0,1.0,1.470600,1314.2200,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
18439,50.0,1.0,11.0,29.0,1.0,0.664940,1460.7800,0.0,1.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225548,43.0,2.0,15.0,23.0,1.0,1.239026,6153.8480,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
225552,42.0,2.0,12.0,22.0,1.0,1.177736,3509.6165,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
225554,24.0,1.0,12.0,6.0,15.0,2.004488,2500.0007,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
225555,51.0,3.0,8.0,35.0,2.0,1.002733,929.0080,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [183]:
logit_model=sm.Logit(y,X)
result = logit_model.fit()
print(result.summary2())

C:\Users\gust\Anaconda3\lib\site-packages\statsmodels\discrete\discrete_model.py:1747: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
C:\Users\gust\Anaconda3\lib\site-packages\statsmodels\discrete\discrete_model.py:1800: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q*np.dot(X,params))))


Optimization terminated successfully.
         Current function value: inf
         Iterations 7


C:\Users\gust\Anaconda3\lib\site-packages\statsmodels\base\model.py:547: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warn('Inverting hessian failed, no bse or cov_params '
C:\Users\gust\Anaconda3\lib\site-packages\statsmodels\base\model.py:547: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warn('Inverting hessian failed, no bse or cov_params '


                                                            Results: Logit
Model:                                       Logit                                    Pseudo R-squared:                         inf   
Dependent Variable:                          on_job                                   AIC:                                      inf   
Date:                                        2021-03-02 16:26                         BIC:                                      inf   
No. Observations:                            40950                                    Log-Likelihood:                           -inf  
Df Model:                                    131                                      LL-Null:                                  0.0000
Df Residuals:                                40818                                    LLR p-value:                              1.0000
Converged:                                   1.0000                                   Scale:                       

## 2.2 Logit 'off job'

In [184]:
import statsmodels.api as sm 

nona = y_X.drop(columns = ["on_job","isic1c_9995", "isic1c_9996", "isic1c_9997", "isic1c_9998", "isic1c_9999"])
nona = nona.dropna()
nona = nona.loc[:, (nona != 0).any(axis=0)]
y = nona["open_educ"]
X = nona.iloc[:, 1:]


In [185]:
logit_model=sm.Logit(y,X)
result = logit_model.fit()
print(result.summary2())

C:\Users\gust\Anaconda3\lib\site-packages\statsmodels\discrete\discrete_model.py:1747: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
C:\Users\gust\Anaconda3\lib\site-packages\statsmodels\discrete\discrete_model.py:1800: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q*np.dot(X,params))))


Optimization terminated successfully.
         Current function value: inf
         Iterations 8


C:\Users\gust\Anaconda3\lib\site-packages\statsmodels\base\model.py:547: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warn('Inverting hessian failed, no bse or cov_params '
C:\Users\gust\Anaconda3\lib\site-packages\statsmodels\base\model.py:547: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warn('Inverting hessian failed, no bse or cov_params '


                                                           Results: Logit
Model:                                      Logit                                    Pseudo R-squared:                         inf   
Dependent Variable:                         open_educ                                AIC:                                      inf   
Date:                                       2021-03-02 16:29                         BIC:                                      inf   
No. Observations:                           40950                                    Log-Likelihood:                           -inf  
Df Model:                                   131                                      LL-Null:                                  0.0000
Df Residuals:                               40818                                    LLR p-value:                              1.0000
Converged:                                  1.0000                                   Scale:                               

## 3. With Penalty

## 3.1. 'on job training'

In [193]:
nona = y_X.drop(columns = ["open_educ","isic1c_9995", "isic1c_9996", "isic1c_9997", "isic1c_9998", "isic1c_9999"])
nona = nona.dropna()
nona = nona.loc[:, (nona != 0).any(axis=0)]
y = nona["on_job"]
X = nona.iloc[:, 1:]

# Split the data into test and training sets, with 30% of samples being put into the test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)



In [197]:
# Create a scaler object
sc = StandardScaler()

# Fit the scaler to the training data and transform
X_train_std = sc.fit_transform(X_train)

# Apply the scaler to the test data
X_test_std = sc.transform(X_test)


In [223]:
C = [10, 1, .1, .001]

for c in C:
    clf = LogisticRegression(penalty='l1', C=c, solver='liblinear')
    clf.fit(X_train, y_train)
    print('C:', c)
    print('Coefficient of each feature:', clf.coef_)
    print('Training accuracy:', clf.score(X_train_std, y_train))
    print('Test accuracy:', clf.score(X_test_std, y_test))
    print('')


C: 10
Coefficient of each feature: [[-1.04368481e-02  1.92304121e-02  9.50046821e-03  1.06000549e-02
   1.02944987e-02 -5.07229738e-02 -1.48678463e-07 -1.40417161e-01
  -2.42469315e-01 -3.06001559e-01 -1.08652613e-01 -2.76163352e-01
   3.20304188e-01 -1.45837964e-01  1.58481169e-01  2.48562986e-02
  -2.33201050e-01 -2.19854406e-01  1.82391609e-01 -4.67329279e-01
  -2.25137597e-01  3.31732231e-01  4.88157246e-02 -3.11929116e-01
   2.84713479e-03 -4.13908415e-02  1.16567660e-01  1.84183615e-01
   5.88569907e-02 -8.63307867e-02 -8.91441659e-01  1.42352909e+00
   5.16027710e-02 -2.04735557e-02 -1.54475758e-01 -3.55781052e-01
   1.26782975e-01 -4.37454496e-01  8.24625531e-03  6.07680679e-02
   1.71693881e-01  3.66224031e+00  4.19180036e+00  4.15593866e+00
   3.89978689e+00  3.73617726e+00  4.17563596e+00  3.95184192e+00
  -1.99270476e-01 -3.60601057e-02 -1.53191843e-01  1.19786095e-04
   2.15773989e-01  1.43182276e-01  5.49397635e-01  9.73722318e-02
  -2.80425900e-01  2.11099076e-02 -1.4830

In [225]:
clf = LogisticRegression(penalty='l1', C=10, solver='liblinear')
logit = clf.fit(X_train, y_train)
print('Coefficient of each feature:', clf.coef_)
#print('Features:', list(X_train.columns))
print('Training accuracy:', logit.score(X_train_std, y_train))
print('Test accuracy:', clf.score(X_test_std, y_test))
print('') 



imp = pd.DataFrame(clf.coef_).T
imp['Feature'] = list(X_train.columns)
imp

Coefficient of each feature: [[-1.18109547e-02  1.13343664e-02  9.76518943e-05  1.04721953e-02
   6.40265240e-03 -5.81723959e-02 -1.45041311e-07 -1.55518163e-01
  -2.58371231e-01 -4.79943131e-01 -2.67626976e-01 -6.86048137e-01
  -4.79141229e-02 -5.36771854e-01 -2.16655024e-01 -3.60111700e-01
  -6.29628275e-01 -6.17200381e-01 -2.11876671e-01 -8.85410475e-01
  -6.01767392e-01 -4.94001407e-02 -3.49933022e-01 -6.97084556e-01
  -4.06658823e-01 -4.37290500e-01 -2.72625420e-01 -2.25437417e-01
  -3.52085878e-01 -5.37058417e-01 -1.33209581e+00  9.77830991e-01
   3.89000347e-02 -3.02934763e-02 -1.99548195e-01 -3.93643843e-01
   9.95852734e-02 -4.80266596e-01 -1.36002146e-02  7.98479221e-02
   1.94625984e-01  3.65416170e+00  4.18853943e+00  4.15357495e+00
   3.90174035e+00  3.72514182e+00  4.16608365e+00  3.92381700e+00
  -1.81725298e-01 -2.46601124e-02 -1.35942530e-01  1.66320178e-02
   2.07121163e-01  1.40664100e-01  3.92664920e-01  9.76264861e-02
  -2.78521732e-01  2.24423435e-02 -1.50177517e-

,0,Feature
0,-0.011811,age_r
1,0.011334,j_q03b
2,0.000098,yrsget
3,0.010472,c_q09
4,0.006403,c_q10a
...,...,...
127,0.063241,g_q05e_Less than once a week but at least once...
128,-0.052173,g_q05e_Never
129,0.088554,skill_1
130,0.166691,skill_2
